# Ingeniería de features de las propiedades en venta.
La idea de este notebook es que comencemos a pensar como procesar los features del Dataset de Properatti que utilizaron para la primer entrega. La ingenieria de features lleva mucho tiempo y trabajo. En esta actividad les presentamos algunas buenas prácticas y les sugerimos algunas ideas, pero queda mucho por hacer! Les invitamos a seguir procesándolo en sus casas :)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
properati = pd.read_csv('datos_properati.csv', delimiter=',', parse_dates = ['created_on'])

In [ ]:
properati.dtypes

## 1- Amigarse con el Dataset

**a)** Indica a qué grupo pertenece las variables de cada columna del Dataset, siendo los grupos:

- Numéricas. 
- Categóricas Nominales.
- Categóricas Ordinales.
- Fechas - Hora - Ubicacion.

Por ejemplo, la columna "lat" tiene variables del tipo " Fechas - Hora - Ubicacion"


In [ ]:
#for col in properati.columns:
#    print(col)
    
for indice, elemento in enumerate(properati.columns):
    print(indice, elemento)

In [ ]:
tipo_variable = [None]*len(properati.columns)
tipo_variable[5] = 'Fechas - Hora - Ubicacion'

for orden,col in enumerate(properati.columns):
    print('{} - En la columna "{}" tenemos variables de tipo "{}" \n'.format(orden,col,tipo_variable[orden]))

**b-** Indica qué columnas tienen NaNs. En caso de tenerlos, cuántos

In [ ]:
# Respuesta
def draw_missing_data_table(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

properati.info()
draw_missing_data_table(properati)

## 2- Manejo de NaNs.
Muchas veces no queremos eliminar las instancias que posean NaNs en algun atributo / feature. 


**a)** Les proponemos utilizar el imputer de Scikit-learn para reemplazar los valores faltantes con algún indicador estadústico. Por ejemplo, en la columna "price_usd_per_m2" podemos reemplazar los valores faltantes por la mediana (les parece correcta esta elección?).

In [ ]:
from sklearn.preprocessing import Imputer
#from sklearn.impute import SimpleImputer
#imp = SimpleImputer(missing_values='NaN', strategy='median', axis=0)
imp = Imputer(missing_values='NaN', strategy='median', axis=0)
## Completar con la instruccion fit_transform. Utilizar como argumento properati[['price_aprox_usd']]
properati_price_imp  = imp.fit_transform(properati[['price_aprox_usd']])

In [ ]:
#print(imp.statistics_)
#draw_missing_data_table(properati)

**b)** Chequear cuántas columnas tiene la variable "properati_price_imp". Tiene algún NaN?

In [ ]:
### Respuesta


**c)** Reemplazar los valores de la columna "properati['price_aprox_usd']"  por los valores de la variable "properati_price_imp".

In [ ]:
### Respuesta

**d)** Responder:
    - Cuáles son ahora las instancias con valores faltantes?
    - Se modificó la mediana de la columna 'price_aprox_usd'? Por qué?
    - Proponer (y NO efectuar) una estrategia para procesar otra columna con valores faltantes.

**e)** Eliminar el resto de las features que tienen valores faltantes. 

## 3- Extrayendo información útil

Los algoritmos de aprendizaje automático que utilizaremos reconocerán patrones en los atributos (o combinaciones de ellos) que les introduzcamos. Entonces, la calidad de funcionamiento del algoritmo esta íntimamente relacionada al preprocesamiento de estos atributos. 


En este caso, las variables del grupo "Fechas - Hora - Ubicacion" pocas veces nos dan la información que queremos y muchas veces hay que reescribirlas. Les proponemos crear **nuevas features** de las cuales sea mas fácil extraer informacion. 


**a)** Les proponemos crear 3 features nuevas, una con el año en que fue publicada cada casa, otra con el mes, y otra con el día. Tomamos como materia prima la información de la columna 'created_on'.

Les mostramos un ejemplo creando la columna "anio", y les proponemos que hagan "mes" y "dias" por su cuenta.

In [ ]:
properati['anio'] = pd.DataFrame((properati['created_on'].map(lambda x: x.year)))


### 4- Manipulación de variables categóricas nominales (o dummies).
Usando LabelEncoder de Scikit-learn podemos transformar nuestras variables categóricas en numéricas sin pasar por variables dummies. Luego, vamos a utilizar OneHotEncoder para transformar las categorías numéricas en categorías binarias.


Nos centraremos en la columna 'property_type'.

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

 
(properati['property_type']).head()

In [ ]:
# Llamamos al algoritmo
le_prop_type = LabelEncoder()
# Lo fiteamos con las categorias que tenemos en nuestro dataset.
le_prop_type.fit(properati['property_type'])

In [ ]:
# Estas son las categorias que eligio el algoritmo. Les asignara un numero a cada categoria, 
# yendo de izquierda a derecha [recordemos que python cuenta desde 0]
le_prop_type.classes_

In [ ]:
#Transformamos la categoria "property_type" en numeros.
cat_prop_type = le_prop_type.transform(properati['property_type'])

In [ ]:
#Les proponemos formas de visualizar que hizo sckit-learn
#print(cat_prop_type)
#plt.plot(cat_prop_type,'o')

In [ ]:
# Actualizamos el viejo dataset
#properati['property_type'] = cat_prop_type

Ahora vamos a transformar las categorías numéricas que acabamos de definir, en binarias. De lo contrario, el algoritmo de ML que vayamos a usar las interpretaria como variables categoricas ordinales [y no lo son!].

In [ ]:
onehot_encoder = OneHotEncoder(sparse = False)
# No olvidemos que hay que cambiarle las dimensiones al set de etrenamiento del OneHotEncoder!
cat_prop_type_rs = cat_prop_type.reshape(-1,1)
print(cat_prop_type_rs)

In [ ]:
#Como estamos cancheres, realizamos directamente el fit y transform juntos!
cat_prop_type_encoded = onehot_encoder.fit_transform(cat_prop_type_rs)
print(cat_prop_type_encoded)

**a)** RESPONDER:

    - Por qué la variable 'cat_prop_type_encoded' tiene esa forma? Qué significa cada cosa?
    - Podriamos haber usado el OneHotEncoder sin utilizar primero el LabelEncoder?

In [ ]:
print(cat_prop_type_encoded)

**b)** Scikit-learn tiene una excelente documentación. Te proponemos que lo chequees por vos misme. Te proponemos que explores https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html. Qué funciones tiene? Qué te parece que hace la función *inverse_transform*?